In [1]:
import json
import pandas as pd

data = []

with open("../data/debate.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [2]:
sample = data[0:10]
sample

[{'id': 't3_2ro9ux',
  'titles': 'Anything that is manmade is natural.',
  'arguments': 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by erriamWebster existing in nature and not made or caused by people coming from nature  as his basis for the distinction for natural vs. unnatural. However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would und

In [3]:
data = []

with open("../data/claim_topics.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
sample = data[0:10]
sample

[{'claim': '. I am insensitive to the Gaming Industrys portrayal of women minorities and how it deals with other socially sensitive topics.',
  'topic': 'discrimination against women'},
 {'claim': '. I believe in same sex and traditional marriage but not other types of marriage',
  'topic': 'mixed marriages'},
 {'claim': '. I believe the BS show Elementary Is worth more praise and acclaim than the BB version Sherlock',
  'topic': 'popularity of the series'},
 {'claim': 'A Wired ouse is better than a Wireless mouse.',
  'topic': 'wireless devices'},
 {'claim': 'A acbook is a better long term investment than a Windows computer.',
  'topic': 'desktop or laptop computers'},
 {'claim': 'A banana is not a good method of portraying scale.',
  'topic': 'scale'},
 {'claim': 'A comic book priced at . or . is disproportionately expensive to the amount of entertainment that can be garnered from it.',
  'topic': 'comic books'},
 {'claim': 'A congressional reboot would breathe new life into the legi

In [28]:
sample_ = sample[4]
sample_["topic"]

'desktop or laptop computers'

In [11]:
import requests

with open("../creds.json", "r") as f:
    config = json.load(f)

In [19]:
params = {
    "topic": str(sample_["topic"]),
    "text": str(sample_["claim"]),
    # "predictedStance": True,
    # "showOnlyArguments": False,
    "userID": config["argumentext_user"],
    "apiKey": config["argumentext_api_key"],
}

url = "https://api.argumentsearch.com/en/classify"

res = requests.get(url, params=params, json=data)
res

<Response [502]>

In [29]:


def identify_stance(topic, claim):
    payload = {
        "topic": topic,
        "text": claim,
        "predictStance": True,
        "userID": config["argumentext_user"],
        "apiKey": config["argumentext_api_key"],
    }

    is_timed_out = True
    timed_out_ctr = 1
    while is_timed_out == True:
        try:
            json_dict = requests.post("https://api.argumentsearch.com/en/classify", timeout=300, data=json.dumps(payload),
                                      headers={'Content-Type': 'application/json'}).json()
            is_timed_out = False
        except requests.exceptions.Timeout or ConnectionError or json.decoder.JSONDecodeError:
            print("Timed out for {0} times.".format(str(timed_out_ctr)))
        except Exception as e:
            print(e)

    return json_dict

res = identify_stance(sample_["topic"], sample_["claim"])
res["sentences"]

[{'argumentConfidence': 0.9952768017537892,
  'argumentLabel': 'argument',
  'sentenceOriginal': 'A acbook is a better long term investment than a Windows computer.',
  'sentencePreprocessed': 'A acbook is a better long term investment than a Windows computer.',
  'sortConfidence': 0.9962138602965112,
  'stanceConfidence': 0.997150918839233,
  'stanceLabel': 'pro'}]